<a href="https://colab.research.google.com/github/Vishwassaini20/final00009/blob/main/BRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Allocation and optimization using tsp

In [ ]:
import pandas as pd
import numpy as np
import folium
import os
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
from google.colab import files
from collections import defaultdict

# Constants
JECRC_LOCATION = (26.78218919094841, 75.82251239614644)  # JECRC College Lat/Lon
BUS_CAPACITY = 60
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Sample Dataset
students_data = [
    [101, "Alice", 26.80018103538852, 75.81106672343208],
    [102, "Bob", 26.804154536290802, 75.8186140767396],
    [107, "Clone", 26.748382466182402, 75.824599053422],
    [103, "Charlie", 26.808527748282295, 75.83163167660328],
    [104, "David", 26.76736069037726, 75.79683527302723],
    [105, "Eve", 26.73718531530984, 75.83157815214602],
    [106, "Vijay", 26.73212055350547, 75.83783435117728],
    [1011, "jay", 26.689034264749157, 75.81634537021407],
    [109, "jake", 26.820399788591445, 75.78369403910385],
    [1010, "jane", 26.808047874709942, 75.81264328240981],
    [112,"Nicky",26.8066332925372, 75.80796568978604],
    [11,"nike",26.81116992505039, 75.82300695749213]

]

bus_routes = {
    "Bus 1": [(26.793034, 75.816592), (26.804090891762804, 75.80850752245559), (26.820120085615468, 75.79687746396749),(26.808982425226095, 75.8316707359115)],
    "Bus 2": [(26.710757660265276, 75.80216642057152), (26.7149863434271, 75.80453338771183), (26.712221453225958, 75.85618182193468)],
    "Bus 3": [(26.766142639799945, 75.80250794110907), (26.778096931594785, 75.79525524754082), (26.764809504402077, 75.79998667838689)],
}

def assign_students_to_buses(student_data, bus_routes):
    assignments = {bus: [] for bus in bus_routes}

    for student in student_data:
        _, name, lat, lon = student
        nearest_bus, nearest_stop, min_distance = None, None, float('inf')

        for bus, stops in bus_routes.items():
            for stop in stops:
                distance = np.linalg.norm(np.array([lat, lon]) - np.array(stop))
                if distance < min_distance and len(assignments[bus]) < BUS_CAPACITY:
                    min_distance = distance
                    nearest_bus, nearest_stop = bus, stop

        if nearest_bus:
            assignments[nearest_bus].append((student[0], name, nearest_stop))

    return assignments

def solve_tsp(distance_matrix):
    """ Solves TSP ensuring the first and last node is JECRC College """
    size = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(size, 1, 0)  # Start at JECRC (index 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return int(distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Force the last stop to be the starting point (JECRC)
    routing.SetFixedCostOfAllVehicles(0)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # Ensure returning to JECRC
        return route
    return []

def create_map_and_image(bus_name, stops, route, students):
    route_map = folium.Map(location=JECRC_LOCATION, zoom_start=12)
    folium.Marker(JECRC_LOCATION, popup='JECRC College', icon=folium.Icon(color='red')).add_to(route_map)

    # Track duplicate locations and apply small offsets
    location_counts = defaultdict(int)

    for student_id, name, stop in students:
        key = tuple(stop)  # Convert location to a hashable key
        location_counts[key] += 1
        lat, lon = stop

        # Apply a small offset if there are multiple students at the same location
        lat_offset = (location_counts[key] - 1) * 0.0001
        lon_offset = (location_counts[key] - 1) * 0.0001

        folium.Marker((lat + lat_offset, lon + lon_offset),
                      popup=f"{name} ({student_id})",
                      icon=folium.Icon(color='blue')).add_to(route_map)

    # Ensure JECRC is the first and last stop
    route_stops = [JECRC_LOCATION] + [stops[i] for i in route if i < len(stops)] + [JECRC_LOCATION]

    folium.PolyLine(route_stops, color='blue', weight=2.5, opacity=1).add_to(route_map)

    file_path_html = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.html")
    route_map.save(file_path_html)
    files.download(file_path_html)

    # Generate PNG Image
    latitudes, longitudes = zip(*route_stops)
    plt.figure(figsize=(8, 6))
    plt.plot(longitudes, latitudes, marker='o', linestyle='-', color='b')
    plt.scatter(*zip(*stops), c='red', marker='x', label='Bus Stops')
    plt.scatter(*JECRC_LOCATION[::-1], c='green', marker='s', label='JECRC College')
    plt.legend()
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Optimized {bus_name} Route')

    file_path_png = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.png")
    plt.savefig(file_path_png)
    plt.close()
    files.download(file_path_png)

def generate_student_allocation_csv(bus_name, students):
    # Create DataFrame for student allocation
    allocation_data = []
    for student_id, name, stop in students:
        allocation_data.append([student_id, name, stop[0], stop[1]])

    df = pd.DataFrame(allocation_data, columns=["Student ID", "Name", "Latitude", "Longitude"])
    file_path_csv = os.path.join(RESULTS_FOLDER, f"{bus_name}_student_allocation.csv")
    df.to_csv(file_path_csv, index=False)
    files.download(file_path_csv)

def main():
    assignments = assign_students_to_buses(students_data, bus_routes)

    for bus_name, students in assignments.items():
        if not students:
            continue

        # Include JECRC College as a start and end point
        assigned_stops = np.array([JECRC_LOCATION] + [s[2] for s in students] + [JECRC_LOCATION])
        distance_matrix = np.linalg.norm(assigned_stops[:, None] - assigned_stops, axis=-1)

        route = solve_tsp(distance_matrix)
        create_map_and_image(bus_name, assigned_stops, route, students)
        generate_student_allocation_csv(bus_name, students)

if __name__ == "__main__":
    main()

#Route map

In [ ]:
import pandas as pd
import numpy as np
import folium
import os
from google.colab import files
from ortools.constraint_solver import routing_enums_pb2, pywrapcp
import openrouteservice

# Constants
JECRC_LOCATION = (26.78218919094841, 75.82251239614644)  # (Latitude, Longitude)
ORS_API_KEY = "5b3ce3597851110001cf6248f7e754b823354dbda5e42be01e776d81"  # OpenRouteService API Key
client = openrouteservice.Client(key=ORS_API_KEY)  # Initialize ORS Client
RESULTS_FOLDER = "results"
os.makedirs(RESULTS_FOLDER, exist_ok=True)

def upload_csv_files():
    """Upload CSV files in Google Colab."""
    uploaded = files.upload()
    return list(uploaded.keys())

def load_bus_csv(file_path):
    """Load the uploaded CSV file and extract student stop coordinates."""
    df = pd.read_csv(file_path)

    # Ensure correct column names
    if 'Latitude' not in df.columns or 'Longitude' not in df.columns:
        print(f"❌ Error: CSV '{file_path}' must have 'Latitude' and 'Longitude' columns.")
        return None

    # Convert to list of tuples (Latitude, Longitude)
    stops = [(row['Latitude'], row['Longitude']) for _, row in df.iterrows()]

    return stops

def solve_tsp(distance_matrix):
    """Solve the Traveling Salesman Problem (TSP) using OR-Tools."""
    size = len(distance_matrix)
    manager = pywrapcp.RoutingIndexManager(size, 1, 0)  # Single vehicle, start at index 0
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return int(distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # Ensure returning to JECRC
        return route
    return []

def get_road_route(ordered_stops):
    """Get real road-based route using OpenRouteService Directions API."""
    try:
        route = client.directions(
            coordinates=[(lon, lat) for lat, lon in ordered_stops],
            profile="driving-car",
            format="geojson"
        )
        return route
    except openrouteservice.exceptions.ApiError as e:
        print(f"❌ Error getting road-based route: {e}")
        return None

def generate_route_map(bus_name, stops, optimized_order):
    """Generate and save a map with the optimized road-based route."""
    route_map = folium.Map(location=JECRC_LOCATION, zoom_start=12)

    # Add Start/End Marker (JECRC College)
    folium.Marker(JECRC_LOCATION, popup='JECRC College', icon=folium.Icon(color='red')).add_to(route_map)

    # Add Student Markers (Blue)
    for lat, lon in stops:
        folium.Marker((lat, lon), popup='Student Stop', icon=folium.Icon(color='blue')).add_to(route_map)

    # Convert optimized order into route path
    ordered_stops = [JECRC_LOCATION] + [stops[i] for i in optimized_order if i < len(stops)] + [JECRC_LOCATION]

    # Get real road-based route from OpenRouteService
    road_route = get_road_route(ordered_stops)

    if road_route:
        folium.PolyLine(
            locations=[(point[1], point[0]) for point in road_route['features'][0]['geometry']['coordinates']],
            color='blue', weight=2.5, opacity=1
        ).add_to(route_map)

    # Save map as HTML
    file_path_html = os.path.join(RESULTS_FOLDER, f"{bus_name}_route.html")
    route_map.save(file_path_html)

    # Download map
    files.download(file_path_html)

def process_bus_routes():
    """Upload and process bus routes, optimizing with TSP + Road-Based Routing."""
    csv_files = upload_csv_files()
    for bus_csv in csv_files:
        stops = load_bus_csv(bus_csv)
        if stops:
            # Compute distance matrix
            all_stops = [JECRC_LOCATION] + stops + [JECRC_LOCATION]
            num_stops = len(all_stops)
            dist_matrix = np.zeros((num_stops, num_stops))

            for i in range(num_stops):
                for j in range(num_stops):
                    dist_matrix[i, j] = np.linalg.norm(np.array(all_stops[i]) - np.array(all_stops[j]))

            # Solve TSP for best route order
            optimized_order = solve_tsp(dist_matrix)

            # Generate and download map
            generate_route_map(bus_csv.split('.')[0], stops, optimized_order)

# Run the function to upload and process bus routes
process_bus_routes()